In [ ]:
#!pip install -U tensorflow-addons
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import nibabel as nib
import glob
import os
import gc
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np
import os
import sys
import random
from skimage.io import imread, imshow
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import time
#import cv2 as cv
from tensorflow.keras.callbacks import *
from tensorflow.python.saved_model import loader_impl # lib for saving models
import plotly.graph_objects as go
#import seaborn as sns
#import pandas as pd
import random; random.seed(2)

In [ ]:
# Esta es una prubea, donde cada slice estara asignada a cada mascara.
def read_images_3D(path,num_images=100 ,show_images= False,img_name='t2', extension='.nii.gz', brain_tumor=[1, 1, 1, 1]):
    #path = 'BraTS2021_Training_Data/'
    # Iterate directory
    folder_list = os.listdir(path)
    print('num of images', len(folder_list))
    flair = '_flair'+extension
    t1    = '_t1'+extension
    t1ce = '_t1ce'+extension
    t2 =   '_t2'+extension #---- Agregar .gz
    mask ='_seg'+extension #---agregar .gz se quita cuando se lee la base de datos de 2020 y 2017
    X = []
    Y = []
    for i in range(1, num_images+1):
        #print(i)
        image_file = folder_list[i] + '/'+ folder_list[i] +'_'+ img_name+ extension#t2
        mask_file = folder_list[i] +'/' + folder_list[i] + mask
        #print(image_file)
        #print(mask_file)
        #print("\n")
        x = nib.load(path+image_file).get_fdata()
        y = nib.load(path+ mask_file).get_fdata()
        y[y == 1] = brain_tumor[0]# 1
        y[y == 2] = brain_tumor[1]#1 #1 # Estaba en 0
        y[y == 3] = brain_tumor[2]#1
        y[y == 4] = brain_tumor[3]#1
        aux = 0
        idx = 0
        idx_img = 0
        n_max = 0
        for j in range(60, 155):
            aux = np.sum(y[:,:,j]>0)
            if aux > n_max:
                idx = j
                n_max = aux
        n_max = 0
        for j in range(60, 155):
            aux = np.sum(x[:,:, j] > 0)
            if aux > n_max:
                idx_img = j
                n_max = aux
        if idx != 0:
            xp = x[:,:, idx]#x[:,:, idx]#dx_img]
            yp = y[:, :, idx]
            yp = yp > 0
            if show_images == True:
                plt.subplot(1, 2, 1),imshow(xp, cmap='gray'), plt.title(image_file)
                plt.subplot(1, 2, 2),plt.imshow(yp, cmap='gray'), plt.title(mask_file)
                plt.show()
            xp =   (xp - np.mean(xp))/np.std(xp) # Statandarization (xp - np.min(xp))/(np.max(xp) - np.min(xp)) #Minmax
            X.append(xp)
            Y.append(yp)
    return X, Y

def read_image_3D(path,  img_name, show_images= False):
    image_file = path+ img_name
    print(image_file)
    img = nib.load(path+image_file).get_fdata()
    return img

In [ ]:
def read_h5_file(PATH, folder, file='train'):
    BRATSNAME = 'BRATS2021'
    PATH_PT_IMAGES = PATH +'/'+ BRATSNAME
    train_images_np = 0
    train_labels_images_np = 0
    if file == 'train':
        with h5py.File(PATH_PT_IMAGES + '/' + BRATSNAME + '_train.h5', 'r') as hf:
            train_images_np = np.array(hf['images'])
        with h5py.File(PATH_PT_IMAGES + '/' + BRATSNAME + '_train_labels.h5', 'r') as hf:
            train_labels_images_np = np.array(hf['images'])  # Aquí cambié a train_labels_images_np

        return train_images_np, train_labels_images_np  # Aquí también uso train_labels_images_np

    elif file == 'test':
        with h5py.File(PATH_PT_IMAGES + '/' + BRATSNAME + '_test.h5', 'r') as hf:
            test_images_np = np.array(hf['images'])
        with h5py.File(PATH_PT_IMAGES + '/' + BRATSNAME + '_test_labels.h5', 'r') as hf:
            test_labels_images_np = np.array(hf['images'])

        return test_images_np, test_labels_images_np
